In [ ]:
import pandas as pd
import torch
import dgl
# Create a DGL graph
G_dgl = dgl.DGLGraph()
num_nodes = nodes_df.shape[0]

# Add nodes with features
G_dgl.add_nodes(num_nodes)

# Set node features
node_features = torch.tensor(nodes_df[['feat1','feat2','feat3','feat4','feat5','feat6','feat7','feat8','feat9','feat10','feat11','feat12','feat13','feat14','feat15','feat16','feat17','feat18','feat19']].values, dtype=torch.float32)
G_dgl.ndata['feat'] = node_features

# Add edges with labels
edges_src = edges_df['src_id'].tolist()
edges_dst = edges_df['dst_id'].tolist()
G_dgl.add_edges(edges_src, edges_dst)

# Set edge types
edge_types = edges_df['label'].astype('category').cat.codes
G_dgl.edata['etype'] = torch.tensor(edge_types.values, dtype=torch.long)

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv

# Define GCN model
class GCNModel(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCNModel, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_feats)
        self.conv2 = GraphConv(hidden_feats, out_feats)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Create a random DGL graph
g = dgl.rand_graph(10, 20)
g.ndata['feat'] = torch.randn(10, 5)

# Initialize the GCN and LSTM models
gcn_model = GCNModel(5, 16, 8)
lstm_model = LSTMModel(8, 32, 1, 6)

# Run GCN to get edge embeddings
node_embeddings = gcn_model(g, g.ndata['feat'])
edge_embeddings = dgl.ops.u_mul_e_sum(g, node_embeddings)

# Prepare edge embeddings for LSTM
edge_embeddings = edge_embeddings.view(1, -1, 8)

# Run LSTM on edge embeddings and update the weights
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)

# Dummy target, replace with actual target
target = torch.tensor([2])  # Represents class d3, with labels ranging from 0 to 5 (inclusive)

for epoch in range(100):
    output = lstm_model(edge_embeddings)
    loss = criterion(output.unsqueeze(0), target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/100], Loss: {loss.item()}")


# not done yet,
